# 1. 어텐션 메커니즘
<br><br>
seq2seq 모델은 인코더에서 입력 시퀀스를 생각벡터라는 하나의 고정된 크기의 벡터 표현으로 압축하고, 디코더는 이 생각벡터를 통해서 출력 시퀀스를 만들어냈다. 이러한 rnn에 기반한 seq2seq모델에는 크게 두 가지 문제가 있다. 
1. 하나의 고정된 크기의 벡터에 모든 정보를 압축하려고 하니까 정보 손실이 발생한다.
2. RNN의 고질적인 문제인 기울기 소실 문제가 존재한다. 
결굴 기계 번역 분야에서 입력 문장이 길면 번역 품질이 떨어지는 현상이 나타난다. 이를 위한 대안으로 입력 시퀀스가 길어지면 시퀀스의 정확도가 떨어지는 것을 보정해주기 위해 등장한 기법이 어텐션이다. 

## 1.1 어텐션 아이디어

기본 아이디어는 디코더에서 출력 단어를 예측하는 매 시점마다, 인코더에서의 전체 입력 문장을 다시 한 번 참고한다는 점이다. **단, 전체 입력 문장을 전부 다 동일한 비율로 참고하는 것이 아니라, 해당 시점에서 예측해야할 단어와 연관이 있는 입력 단어 부분을 좀 더 집중attention!! 해서 보게 된다.**

## 1.2 어텐션 함수

<key-value를 구성하는 딕셔너리 자료형>
**dict = {'2017':'transformer', '2018':'Bert'}

<img src="./image/어텐션함수.jpg" width="500" height="200">

어텐션을 함수로 표현하면 **Attention(Q, K, V) = Attention Value**
어텐션 함수는 주어진 query에 대해서 모든 key와의 유사도를 각각 구한다. 구해낸 이 유사도를 키와 맵핑되어있는 각각의 value에 반영해준다. 그리고 유사도가 반영된 value를 모두 더해서 리턴한다. 여기서는 이를 어텐션 값value 라고 하겠다. 

Q = Query : t 시점의 디코더 셀에서의 은닉 상태

K = Keys : 모든 시점의 인코더 셀의 은닉 상태들

V = Values : 모든 시점의 인코더 셀의 은닉 상태들

## 1.3 Dot-Product Attention
<br>
수식적으로 이해하기 쉽게 수식을 적용한 닷-프로덕트 어텐션을 통해 어텐션을 이해해보자. 
<img src="./image/닷프로덕트.jpg" width="500" height="200">

디코더의 세번째 LSTM셀에서 출력 단어를 예측할 때, 어텐션 메커니즘을 사용하는 모습을 보여준다. 디코더의 첫번재, 두번째 LSTM셀은 이미 어텐션 메커니즘을 통해 je와 suis를 예측하는 과정을 거쳤다고 가정한다. **디코더의 세번째 LSTM 셀은 출력 단어를 예측하기 위해서 인코더의 모든 입력 단어들의 정보를 다시 한번 참고하고자 한다.** 이때 소프트맥스 함수의 결과값은 I,am,a,student 단어 각각이 출력 단어를 예측할 때 얼마나 도움이 되는지의 정도를 수치화한 값이다. 위 그림에서는 빨간 직사각형 크기로 소프트맥스 함수의 결과값 크기를 표현하였다. 즉, 직사각형 크기가 클 수록 도움이 되는 정도가 크다.각 입력 단어가 디코더의 예측에 도움이 되는 정도를 수치화하여 측정되면 이를 하나의 정보로 담아서 디코더로 전송된다.  
<br><br>

# 2. attention vlaue 구하기
<br>

앞서, 디코더의 현재 시점 t에서 출력 단어를 예측하기 위해서는 디코더의 셀은 두 개의 입력값을 필요로 한다. **바로 이전 시점인 t-1의 은닉 상태와 이전 시점 t-1에 나온 출력 단어이다.** 그런데 어텐션 메커니즘에서는 어텐션 값이라는 새로운 값을 필요로 하는데 이 이후의 단계는 모두 이 어텐션 값을 구하기 위한 단계이다. 

## 2.1 어텐션 스코어 Attention Score를 구한다.**

어텐션 스코어란 현재 디코더의 시점 t에서 단어를 예측하기 위해, 인코더의 모든 은닉 상태 각각이 디코더의 현 시점의 은닉 상태 st와 얼마나 유사한지를 판단하는 스코어값이다. 닷-프로덕트 어텐션에서는 이 스코어 값을 구하기 위해, st를 전치하고 각 은닉 상태와 내적을 수행한다. 즉, 모든 어켄션 스코어 값은 스칼라이다.  
<img src="./image/어텐션스코어.jpg" width="500" height="200">
<img src="./image/어텐션스코어함수.jpg" width="150" height="100">
<br>
## 2.2 소프트맥스 함수를 통해 어텐션 분포를 구한다.**

et에 소프트맥스 함수를 적용하여, 모든 값을 합하면 1이 되는 확률 분포를 얻어낸다. 이를 어텐션 분포라고 하며, 각각의 값은 어텐션 가중치라고 한다. 
<img src="./image/어텐션분포.jpg" width="500" height="200">
디코더의 시점 t에서의 어텐션 가중치의 모음값인 어텐션 분포를 αt이라고 할 때
<img src="./image/어텐션분포함수.jpg" width="150" height="100">

## 2.3 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값을 구한다. 

지금까지 준비해온 정보들을 하나로 합치는 단계이다. 각 인코더의 은닉 상태와 어텐션 가중치값들을 곱하고, 최종적으로 모두 더한다. 
<img src="./image/어텐션값.jpg" width="500" height="200">
<img src="./image/어텐션값함수.jpg" width="150" height="100">
이러한 어텐션 값은 인코더의 문맥을 포함하고 있다고 하여, context vevtor라고도 불린다. 기본적인 seq2seq에서 인코더의 마지막 은닉 상태를 컨텍스트 벡터라고 부르는 것과 대조된다. 

## 2.4 어텐션 값과 디코더의 t 시점의 은닉 상태를 연결한다.

어텐션 값이 구해지면 어텐션 메커니즘은 알파티와 에스티를 결합하여 하나의 벡터로 만드는 작업을 수행한다. 이를 vt라고 정의해보겠다. 그리고 vt를 예측 연산의 입력으로 사용하므로서 인코더로부터 얻은 정보를 활용하여 더 잘 예측할 수 있게 된다. 이것이 어텐션 메커니즘의 핵심이다. 
<img src="./image/은닉상태연결.jpg" width="500" height="200">

## 2.5 출력층 연산의 입력이 되는 ~st를 계산한다.

<img src="./image/연산.jpg" width="500" height="200">
논문에서는 vt를 바로 출력층으로 보내기전에 연산을 한 번 더 추가했다. 가중치 행렬과 곱한후에 하이퍼볼릭탄젠트 함수를 지나도록 하여 출력층 연산을 위한 새로운 벡터인 ~st를 얻었다. 

## 2.6 ~St를 출력층의 입력으로 사용한다. 

<img src="./image/출력.jpg" width="200" height="100">

- 지금 보인 닷-프로덕트 어텐션의 이름은 어텐션 스코어를 구하는 방법이 내적이었기 때문이다. 닷 프로덕트 어텐션과 다른 어텐션들의 차이는 중간 수식의 차이이다. 
- 어텐션 스코어를 구하는 방법은 여러가지가 제시되어 있으며, dot, scaled dot, general, concat, location-base등이 있다. 

<br><br>
# 3. 바다나우 어텐션 Bahdanau

**Attention(Q, K, V) = Attention Value**

- t = 어텐션 메커니즘이 수행되는 디코더 셀의 현재 시점을 의미.
- Q = Query : t-1 시점의 디코더 셀에서의 은닉 상태
- K = Keys : 모든 시점의 인코더 셀의 은닉 상태들
- V = Values : 모든 시점의 인코더 셀의 은닉 상태들

여기서 어텐션 함수의 Query가 디코더 셀의 t시점의 은닉 상태가 아니라 t-1시점의 은닉 상태임을 주의하자. 

## 3.1 어텐션 스코어를 구한다. 

<img src="./image/바다나우어텐션스코어.jpg" width="500" height="200">

앞에 루옹 어텐션은 Query로 디코더의 t 시점의 은닉 상태를 사용한 것과는 달리 이번에는 t-1 시점의 은닉 상태 St-1를 사용한다. 바다나우 어텐션의 어텐션 스코어 함수는 

<img src="./image/바다나우함수.jpg" width="300" height="150">

<tanh 함수 인자 두개에 대해서>

<img src="./image/탄에이치함수인자.jpg" width="500" height="200">

W는 학습 가능한 가중치 행렬이다. 

<위의 저 두 행렬을 더한 후, 하이퍼볼릭탄젠트 함수를 지나도록 한다.>

<img src="./image/탄에이치.jpg" width="500" height="200">

<마지막으로 가중치 Wa와 곱해서 St-1와 인코더의 각 은닉층의 유사도가 기록된 어텐션 스코어 벡터를 얻을 수 있다.>

<img src="./image/어텐션스코어산출.jpg" width="500" height="200">

## 3.2 소프트맥스 함수를 통해 어텐션 분포를 구한다. 

위에서 구한 어텐션 스코어 벡터에 소프트맥스 함수를 적용하여, 모든 값을 합하면 1이 되는 확률 분포를 얻는다. 이를 어텐션 분포라고 하며, 각각의 값은 어텐션 가중치라고 한다.

<img src="./image/어텐션가중치.jpg" width="500" height="200">

## 3.3 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값!!을 구한다.

어텐션의 최종 결과값을 얻기 위해 각 인코더의 은닉 상태와 어텐션 가중치값들을 곱하고, 최종적으로 모두 더한다. 이 벡터는 인코더의 문맥을 포함하고 있다고하여, context vector라고 부른다. 
<img src="./image/바우나우어텐션값.jpg" width="500" height="200">

## 3.4 컨텍스트 벡터로 부터 St를 구한다. 

<기존 LSTM이 St를 구할 때>

이전 시점의 셀로부터 전달받은 은닉 상태St-1와 현재 시점의 입력 Xt를 가지고 연산. 

<어텐션 메커니즘>

바다나우 어텐션 메커니즘에서는 컨텍스트 벡터와 현재 시점의 입력인 단어 임베딩 벡터를 연결하고, 현재 시점의 새로운 입력으로 사용한다. 그리고 이전 시점의 셀로부터 전달받은 은닉 상태St-1와 현재 시점의 새로운 입력으로부터 St를 구한다. **기존의 LSTM이 임베딩된 단어 벡터를 입력으로 하는 것에서 컨텍스트 벡터와 임베딩된 단어 벡터를 연결하여 입력으로 사용하는 것이 달라졌다.**

<img src="./image/바다나우출력.jpg" width="500" height="200">

<br><br>
# 4. 어텐션 메커니즘 실습

## 4.1 IMDB 리뷰 데이터 전처리

In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000 # 최대 단어 개수를 10,000로 제한
(X_train,y_train),(X_test,y_test) = imdb.load_data(num_words = vocab_size)

IMDB 데이터는 이미 정수 인코딩이 된 상태이므로 남은 전처리는 패딩뿐이다.

In [2]:
print(X_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [3]:
print('리뷰 최대 길이: {}'.format(max(len(review) for review in X_train)))
print('리뷰 평균 길이: {}'.format(sum(map(len,X_train)) / len(X_train)))

리뷰 최대 길이: 2494
리뷰 평균 길이: 238.71364


In [4]:
maxlen = 500
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [5]:
len(X_train[0])

500

## 4.2 바다나우 어텐션 Bahdanau Attention

텍스트 분류에서 어텐션 메커니즘을 사용하는 이유가 무엇일까? RNN의 마지막 은닉 상태는 예측을 위해 사용된다. 그런데 이 RNN의 마지막 은닉 상태는 몇 가지 유용한 정보들을 손실한 상태이다. 그래서 RNN이 time step을 지나며 손실했던 정보들을 다시 참고하고자 하는 것이다. **RNN의 모든 은닉 상태들을 다시 한번 참고하겠다는 것이다.**

다음은 바다나우 어텐션의 어텐션 스코어 함수를 구현한것이다. 

<img src="./image/바다나우실습.jpg" width="300" height="150">

- Query : t-1 시점의 디코더 셀에서의 은닉 상태
- Keys : 모든 시점의 인코더 셀의 은닉 상태들

In [5]:
import tensorflow as tf

class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

## 4.3 Modeling + Attention  - 케라스 함수형 API

In [6]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input,Model
from tensorflow.keras import optimizers

In [7]:
sequence_input = Input(shape=(maxlen,), dtype='int32')

# 10,000개의 단어들을 128차원 벡터로 임베딩하겠다. 
embedded_sequences = Embedding(vocab_size, 128, input_length=maxlen, mask_zero = True)(sequence_input)


# 양방향 LSTM을 두 층으로 사용
# 첫번째 층, 이후 두번 째 층을 쌓을 예정이므로 return_sequences를 True로 해주어야함
lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True))(embedded_sequences)

lstm, forward_h, forward_c, backward_h, backward_c = \
Bidirectional(LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

In [8]:
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)


양방향 LSTM을 사용할 경우 순방향 LSTM과 역방향 LSTM 각각 은닉 상태와 셀 상태를 가지므로, **양방향 LSTM의 은닉 상태와 셀 상태를 사용하려면 두 방향의 LSTM의 상태들을 연결해주면 된다.** 

In [9]:
state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

attention = BahdanauAttention(64) # 가중치 크기 정의

# 어텐션 메커니즘에서는 은닉 상태를 사용한다. state_h를 입력으로 컨텍스트 벡터를 얻음
context_vector, attention_weights = attention(lstm, state_h)

In [10]:
# 위의 컨텍스트 벡터를 밀집층에 통과시키고, 
#이진 분류이므로 최종 출력층에 1개의 뉴련 배치, 활성화 함수로 시그모이드 함수를 사용
dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs=output)

In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs = 1, batch_size = 256, validation_data=(X_test, y_test), verbose=1)

Train on 25000 samples, validate on 25000 samples
 1536/25000 [>.............................] - ETA: 6:14:11 - loss: 0.6925 - accuracy: 0.5267

학습이 너무 너무 오래걸려서 일시중지ㅠㅠ